<a href="https://colab.research.google.com/github/Phananhne/BaiTapAndroind/blob/master/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ConvLab/ConvLab-3.git
%cd ConvLab-3
!pip install -e .


fatal: destination path 'ConvLab-3' already exists and is not an empty directory.
/content/ConvLab-3
Obtaining file:///content/ConvLab-3
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for convlab (pyproject.toml) ... done
  Created wheel for convlab: filename=convlab-3.0.2b0-py3-none-any.whl size=10662 sha256=f95dba64296bfb5eaf58e7a3e66c1bc5112172a2161e864faf55009ee7d0b9e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-mtts4rly/wheels/8a/a2/ec/b322e7a4e0b1006838d79ba4f7c69a65c04d263ce01bc28157
Successfully built convlab
  Attempting uninstall: convlab
    Found existing installation: convlab 3.0.2b0
    Uninstalling convlab-3.0.2b0:
      Successfully uninstalled convlab-3.0.2b0


In [3]:
# 2️⃣ Cài thêm ConvLab-2 (để có module jointBERT)
%cd /content
!git clone https://github.com/ConvLab/ConvLab-2.git
%cd ConvLab-2
!pip install -e .


/content
Cloning into 'ConvLab-2'...
remote: Enumerating objects: 443, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 443 (delta 10), reused 11 (delta 6), pack-reused 416 (from 1)
Receiving objects: 100% (443/443), 113.61 MiB | 29.27 MiB/s, done.
Resolving deltas: 100% (146/146), done.
/content/ConvLab-2
Obtaining file:///content/ConvLab-2
ERROR: file:///content/ConvLab-2 does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [4]:
# 3️⃣ Thêm cả hai vào Python path
import sys
sys.path.append('/content/ConvLab-3')
sys.path.append('/content/ConvLab-2')

In [6]:
!pip install unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.4 MB/s eta 0:00:00


In [7]:
# available NLU models
# from convlab.nlu.svm.multiwoz import SVMNLU
from convlab.policy.dqn.multiwoz.dqn_policy import DQNPolicy
from convlab.nlu.jointBERT.multiwoz import BERTNLU
# from convlab.nlu.milu.multiwoz import MILU
# available DST models
from convlab.dst.rule.multiwoz import RuleDST
# from convlab.dst.mdbt.multiwoz import MDBT
# from convlab.dst.sumbt.multiwoz import SUMBT
# from convlab.dst.trade.multiwoz import TRADE
# from convlab.dst.comer.multiwoz import COMER
# available Policy models
from convlab.policy.rule.multiwoz import RulePolicy
# from convlab.policy.ppo.multiwoz import PPOPolicy
# from convlab.policy.pg.multiwoz import PGPolicy
# from convlab.policy.mle.multiwoz import MLEPolicy
# from convlab.policy.vhus.multiwoz import UserPolicyVHUS
# from convlab.policy.mdrg.multiwoz import MDRGWordPolicy
# from convlab.policy.hdsa.multiwoz import HDSA
# from convlab.policy.larl.multiwoz import LaRL
# available NLG models
from convlab.nlg.template.multiwoz import TemplateNLG
from convlab.nlg.sclstm.multiwoz import SCLSTM
# available E2E models
# from convlab.e2e.sequicity.multiwoz import Sequicity
# from convlab.e2e.damd.multiwoz import Damd
from convlab.dialog_agent import PipelineAgent, BiSession
from convlab.evaluator.multiwoz_eval import MultiWozEvaluator
from convlab.util.analysis_tool.analyzer import Analyzer
from pprint import pprint
import random
import numpy as np
import torch

In [8]:
def set_seed(r_seed):
    random.seed(r_seed)
    np.random.seed(r_seed)
    torch.manual_seed(r_seed)

In [9]:
def test_end2end():
    # go to README.md of each model for more information
    # BERT nlu
    sys_nlu = BERTNLU()
    # simple rule DST
    sys_dst = RuleDST()
    # rule policy
    sys_policy = DQNPolicy()
    # template NLG
    sys_nlg = TemplateNLG(is_user=False)
    # assemble
    sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, name='sys')

    # BERT nlu trained on sys utterance
    user_nlu = BERTNLU(mode='sys', config_file='multiwoz_sys_context.json',
                       model_file='https://huggingface.co/ConvLab/ConvLab-2_models/resolve/main/bert_multiwoz_sys_context.zip')
    # not use dst
    user_dst = None
    # rule policy
    user_policy = RulePolicy(character='usr')
    # template NLG
    user_nlg = TemplateNLG(is_user=True)
    # assemble
    user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

    analyzer = Analyzer(user_agent=user_agent, dataset='multiwoz')

    set_seed(20200202)
    analyzer.comprehensive_analyze(sys_agent=sys_agent, model_name='BERTNLU-RuleDST-DQNPolicy-TemplateNLG', total_dialog=1000)

In [10]:

if __name__ == '__main__':
    test_end2end()

load train, size 8434
load val, size 999
load test, size 1000
loaded train, size 113500
loaded val, size 14730
loaded test, size 14744
dialog act num: 36
sentence label num: 137
tag num: 331


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Load actions from data..


Dimension of system actions: 208
Dimension of user actions: 79
State dimension: 361
NLG seed 0
load train, size 8434
load val, size 999
load test, size 1000
loaded train, size 56750
loaded val, size 7365
loaded test, size 7372
dialog act num: 34
sentence label num: 65
tag num: 319
Load from https://huggingface.co/ConvLab/ConvLab-2_models/resolve/main/bert_multiwoz_sys_context.zip


100%|██████████| 425232234/425232234 [00:08<00:00, 52898819.46B/s]


bert-base-uncased
Loading goal model is done
NLG seed 0


TypeError: Population must be a sequence.  For dicts or sets, use sorted(d).